In [4]:
# Instalar librerias y cargar modulos
!pip -q install plotly ipywidgets

import os
import sqlite3
import pandas as pd
import plotly.express as px
from google.colab import files

# Definir rutas
CSV_PATH = "/content/spotify_churn_dataset.csv"
DB_PATH = "/content/spotify.db"

In [5]:
# Crear la base de datos y la tabla en SQLite
schema_sql = """
CREATE TABLE IF NOT EXISTS spotify_users (
    user_id INTEGER PRIMARY KEY,
    gender TEXT,
    age INTEGER,
    country TEXT,
    subscription_type TEXT,
    listening_time INTEGER,
    songs_played_per_day INTEGER,
    skip_rate REAL,
    device_type TEXT,
    ads_listened_per_week INTEGER,
    offline_listening INTEGER, -- 0/1
    is_churned INTEGER         -- 0/1
);
"""

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()
cur.executescript(schema_sql)
conn.commit()
conn.close()

In [6]:
# Cargar el CSV en la base de datos
# Leer el CSV
df_csv = pd.read_csv(CSV_PATH)

# Insertar en SQLite
conn = sqlite3.connect(DB_PATH)
df_csv.to_sql("spotify_users", conn, if_exists="replace", index=False)
conn.close()

print(f"{len(df_csv)} registros cargados en la tabla spotify_users")

8000 registros cargados en la tabla spotify_users


In [7]:
# Consultar datos desde la base
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql("SELECT * FROM spotify_users", conn)
conn.close()

df.head()

,user_id,gender,age,country,subscription_type,listening_time,songs_played_per_day,skip_rate,device_type,ads_listened_per_week,offline_listening,is_churned
0,1,Female,54,CA,Free,26,23,0.20,Desktop,31,0,1
1,2,Other,33,DE,Family,141,62,0.34,Web,0,1,0
2,3,Male,38,AU,Premium,199,38,0.04,Mobile,0,1,1
3,4,Female,22,CA,Student,36,2,0.31,Mobile,0,1,0
4,5,Other,29,US,Family,250,57,0.36,Mobile,0,1,1


In [16]:
# Visualizaciones con Plotly
tmp = df["subscription_type"].value_counts().reset_index()
tmp.columns = ["Tipo de suscripción", "Cantidad"]

fig1 = px.bar(tmp, x="Tipo de suscripción", y="Cantidad",
              title="Distribución de usuarios por tipo de suscripción",
              labels={"Cantidad":"Cantidad de usuarios"})
fig1.update_layout(xaxis={'categoryorder':'total descending'})  # ordena de mayor a menor
fig1.show()


# Churn por suscripción
tmp = df.groupby(["subscription_type","is_churned"]).size().reset_index(name="Cantidad")
tmp["is_churned"] = tmp["is_churned"].map({0:"Activo", 1:"Canceló"})

fig2 = px.bar(tmp, x="subscription_type", y="Cantidad", color="is_churned",
              barmode="group",
              title="Churn (cancelaciones) por tipo de suscripción",
              labels={"subscription_type":"Tipo de suscripción", "is_churned":"Estado"})
fig2.show()

# Tasa de churn total
churn_counts = df["is_churned"].map({0:"Activo", 1:"Canceló"}).value_counts().reset_index()
churn_counts.columns = ["Estado", "Cantidad"]

fig_pie_churn = px.pie(churn_counts, names="Estado", values="Cantidad",
                       title="Proporción de usuarios activos vs cancelados")
fig_pie_churn.update_traces(textposition="inside", textinfo="percent+label")
fig_pie_churn.show()


# Porcentaje por tipo de dispositivo (torta)
counts = df["device_type"].value_counts().reset_index()
counts.columns = ["Dispositivo", "Cantidad"]

fig_pie = px.pie(counts, names="Dispositivo", values="Cantidad",
                 title="Porcentaje por tipo de dispositivo")
# Mostrar etiqueta + porcentaje dentro de cada porción
fig_pie.update_traces(textposition="inside", textinfo="percent+label")
fig_pie.show()

# Filtros
from ipywidgets import interact, Dropdown

@interact(
    pais=Dropdown(options=["Todos"] + sorted(df["country"].unique().tolist()), value="Todos", description="País:"),
    suscripcion=Dropdown(options=["Todos"] + sorted(df["subscription_type"].unique().tolist()), value="Todos", description="Suscripción:")
)
def ver_filtro(pais, suscripcion):
    dff = df.copy()
    if pais != "Todos":
        dff = dff[dff["country"] == pais]
    if suscripcion != "Todos":
        dff = dff[dff["subscription_type"] == suscripcion]

    dff["Estado"] = dff["is_churned"].map({0:"Activo", 1:"Canceló"})

    fig = px.histogram(dff, x="subscription_type", color="Estado",
                       barmode="group",
                       title=f"Usuarios filtrados por país ({pais}) y suscripción ({suscripcion})",
                       labels={"subscription_type":"Tipo de suscripción", "Estado":"Estado de usuario"})
    fig.show()


interactive(children=(Dropdown(description='País:', options=('Todos', 'AU', 'CA', 'DE', 'FR', 'IN', 'PK', 'UK'…

In [17]:
# Descargar base
from google.colab import files
files.download(DB_PATH)  # Te descarga spotify.db a tu PC

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>